In [ ]:
pip install numpy pandas tensorflow

In [ ]:
import pandas as pd
import numpy as np

# Load the data
data = pd.read_csv('/content/VAL_Q3_Save your spot_Page_3.csv')

target_col = 'Event count'

print(data.isnull().sum())
data[target_col].fillna(0, inplace=True)
print(data.isnull().sum())

# Data preprocessing
# Since LSTM requires sequence data, we'll use a time window to create sequences.
def create_sequences(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(X), np.array(y)

# Extract the 'Views' column as the target variable to forecast
data = data[target_col].values.reshape(-1, 1)

# Normalize the data (optional, but recommended for better model performance)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = scaler.fit_transform(data)

# Choose the time window size for creating sequences
window_size = 30  # You can experiment with different window sizes

# Create sequences for training
X, y = create_sequences(data_normalized, window_size)

# Split the data into training and testing sets
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


Date           0
Event count    1
dtype: int64
Date           0
Event count    0
dtype: int64


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Create the LSTM model
model = Sequential()
model.add(LSTM(units=64, input_shape=(window_size, 1)))
model.add(Dense(units=1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Print the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                16896     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 16961 (66.25 KB)
Trainable params: 16961 (66.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Train the model
epochs = 100  # You can experiment with different numbers of epochs
batch_size = 32  # You can experiment with different batch sizes

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)


Epoch 1/100
7/7 [==============================] - 3s 32ms/step - loss: 0.0394
Epoch 2/100
7/7 [==============================] - 0s 24ms/step - loss: 0.0207
Epoch 3/100
7/7 [==============================] - 0s 29ms/step - loss: 0.0179
Epoch 4/100
7/7 [==============================] - 0s 26ms/step - loss: 0.0181
Epoch 5/100
7/7 [==============================] - 0s 27ms/step - loss: 0.0174
Epoch 6/100
7/7 [==============================] - 0s 24ms/step - loss: 0.0175
Epoch 7/100
7/7 [==============================] - 0s 26ms/step - loss: 0.0174
Epoch 8/100
7/7 [==============================] - 0s 23ms/step - loss: 0.0174
Epoch 9/100
7/7 [==============================] - 0s 31ms/step - loss: 0.0173
Epoch 10/100
7/7 [==============================] - 0s 25ms/step - loss: 0.0172
Epoch 11/100
7/7 [==============================] - 0s 28ms/step - loss: 0.0172
Epoch 12/100
7/7 [==============================] - 0s 25ms/step - loss: 0.0171
Epoch 13/100
7/7 [==============================]

In [ ]:
# Evaluate the model on the testing data
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

# Add the code to compare test values and predicted values and calculate MAPE here
# Make predictions on the test data
y_pred = model.predict(X_test)

# Inverse transform the test data and predictions to their original scales
y_test_original = scaler.inverse_transform(y_test)
y_pred_original = scaler.inverse_transform(y_pred)

# Create a DataFrame to compare test values and predicted values
comparison_df = pd.DataFrame({'Actual': y_test_original.ravel(), 'Predicted': y_pred_original.ravel()})

# Calculate the Mean Absolute Percentage Error (MAPE)
def calculate_mape(actual, predicted):
    return (1/len(actual)) * np.sum(np.abs((actual - predicted) / actual)) * 100

mape = calculate_mape(y_test_original, y_pred_original)

# Print the comparison DataFrame and MAPE
print(comparison_df)
print(f'MAPE: {mape:.2f}%')

In [ ]:
import pandas as pd

# Assuming you have the date range from April 1, 2023, to June 30, 2023
date_range = pd.date_range(start='2023-07-01', periods=3, freq='M')

# Forecast the next three months (assuming the last window_size data points as the starting point)
last_window = data_normalized[-window_size:].reshape(1, window_size, 1)
forecasts_normalized = []

# Forecast each month separately and store the results in a list
for i in range(3):
    forecast_normalized = model.predict(last_window)
    forecasts_normalized.append(forecast_normalized[0, 0])
    last_window = np.append(last_window[:, 1:, :], forecast_normalized.reshape(1, 1, 1), axis=1)

# Inverse transform the forecasted values to get the actual scale
forecasts = scaler.inverse_transform(np.array(forecasts_normalized).reshape(-1, 1))

# Create a DataFrame to store the predictions
predictions_df = pd.DataFrame({'Date': date_range, 'Predicted Event Count': forecasts.ravel()})

# Print the DataFrame
print(predictions_df)


1/1 [==============================] - 0s 106ms/step
        Date  Predicted Event Count
0 2023-07-31              36.058701
1 2023-08-31              38.167961
2 2023-09-30              39.078854
